In this notebook we check the simplifier solver for random unitary matrices (2-qubit). In the first part of the notebook we see that the method we already have akes (at maximum) ~3 sec for 2000 random unitaries; while for future work we leave mp.mathtool. THe problem i was having was the np.round !

In [9]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sympy 
from datetime import datetime
from scipy import stats

def symp_solve(t,a,b,g):
    return sympy.nsolve(t,[a,b,g],np.pi*np.array([np.random.random(),np.random.random(),np.random.random()]) ,maxsteps=3000, verify=True)

rrs=0
us=0
times=[]
while True:
    us +=1
    error = True
    st=datetime.now()
    while error is True:
        now = (datetime.now()-st).total_seconds()
        
        u = stats.unitary_group.rvs(2)
        a = sympy.Symbol("a")
        b = sympy.Symbol("b")
        g = sympy.Symbol("g")
        eqs = [sympy.exp(-sympy.I*.5*(a+g))*sympy.cos(.5*b) ,
               -sympy.I*sympy.exp(-sympy.I*.5*(a-g))*sympy.sin(.5*b),
                sympy.exp(sympy.I*.5*(a+g))*sympy.cos(.5*b)
              ]
        kk = np.reshape(u, (4,))
        s=[]
        for i,r in enumerate(kk):
            if i!=2:
                s.append(r)
        t=[]
        for eq, val in zip(eqs,s):
            t.append(eq-val)
        try:
            solution = symp_solve(t,a,b,g)
            vals = np.array(solution.values()).astype(np.complex64)
            error=False
            times.append((datetime.now()-st).total_seconds())
        except Exception:
            error=True
            rrs+=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


KeyboardInterrupt: 

In [12]:
print(len(times),np.max(times))

845 0.280173


Some attemps to write this as mp

In [19]:
import mpmath as mp

In [18]:
f = [lambda b: mp.exp(-1j*b)*mp.cos(b)]
mp.findroot(f[0], (np.pi/2))

mpc(real='1.5707963267948966', imag='9.0851626059990175e-45')

In [16]:
f = [lambda a,b: mp.exp(-1j*(b+a))*mp.cos(b)*mp.cos(a), lambda a,b: mp.exp(-1j*(b+a))*mp.cos(b)*mp.sin(a)]
mp.findroot(f, (np.pi/2,np.pi/2))

matrix(
[[mpc(real='1.5707963267948966', imag='-1.4816541294525403e-49')],
 [mpc(real='1.5707963267948966', imag='8.9104407089469785e-28')]])

In [22]:
f = [lambda b,g,d: mp.exp(-0.5j*(b+d))*mp.cos(0.5*g), lambda b,g,d: mp.exp(-0.5j*(a-g))*mp.sin(0.5*b), 
     lambda b,g,d: mp.exp(0.5j*(b+d))*mp.cos(0.5*g)]

mp.findroot(f, (np.pi/2,np.pi/2,np.pi/1))

TypeError: <lambda>() missing 2 required positional arguments: 'g' and 'd'

An attempt to write this as variational problem

In [ ]:


def loss(pred, target):
    #pred ---> tf ragged tensor
    return 1-0.25*np.trace(np.real(np.dot(np.conjugate(np.transpose(np.squeeze(pred.numpy()))), target))**2)


class Ucompiler(tf.keras.Model):
    def __init__(self):
        """
        symbols: symbolic variable [sympy.Symbol]*len(rotations_in_circuit)
        batch_size: how many circuits you feed the model at, at each call (this might )
        """
        super(Ucompiler,self).__init__()
        self.unitary_layer = tfq.layers.Unitary()
        self.tvars = [tf.random.uniform([3])*0.]
        gts=[cirq.rx, cirq.rz,cirq.rx]
        symbols_names = [str(k) for k in range(3)]
        self.symbols = [sympy.Symbol("{}".format(k)) for k in range(3)]
        cirq.Circuit([g(sym).on(qubit) for g,sym in zip(gts, symbols)])
        qubit = cirq.GridQubit.rect(1, 1)[0]
        self.circuit = cirq.Circuit([g(sym).on(qubit) for g,sym in zip(gts, symbols)])
        self.example_resolver = {s:k for s,k in zip(symbols_names, np.random.randn(len(symbols)))}
        
    def call(self,dummy):
        return self.unitary_layer(self.circuit, symbol_names=list(self.example_resolver.keys()), symbol_values=self.tvars)


utr.unitary_layer(utr.circuit, symbol_names=list(utr.example_resolver.keys()), symbol_values=utr.tvars)




circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
output = tfq.layers.Unitary()(circuit_input, symbol_names=[str(k) for k in range(3)], symbol_values=[[1,2,3]])
model = tf.keras.Model(inputs=circuit_input, outputs=output)
model.compile(loss="mse", optimizer="adam")







gts=[cirq.rx, cirq.rz,cirq.rx]
symbols_names = [str(k) for k in range(3)]
symbols = [sympy.Symbol("{}".format(k)) for k in range(3)]
circuit = cirq.Circuit([g(sym).on(qubit) for g,sym in zip(gts, symbols)])


q = cirq.GridQubit(0, 0)

(a, b, c) = sympy.symbols("a b c")
circuit = cirq.Circuit(
    cirq.rx(a)(q),
    cirq.rz(b)(q),
    cirq.rx(c)(q),
)


outputs = tfq.layers.PQC(circuit, cirq.Z(q))
quantum_data = tfq.convert_to_tensor([
    cirq.Circuit(),
])
#measurement = [cirq.X(q), cirq.Y(q), cirq.Z(q)]















class Ucompiler(tf.keras.Model):
    def __init__(self, symbols, observable, batch_sizes=1):
        """
        symbols: symbolic variable [sympy.Symbol]*len(rotations_in_circuit)
        batch_size: how many circuits you feed the model at, at each call (this might )
        """
        super(Ucompiler,self).__init__()
        self.gradient_norm = Metrica(name="grad_norm")
        
    def call(self, inputs):
        """
        inputs: tfq circuits (resolved or not, to train one has to feed unresolved)
        """
        return self
        f = self.expectation_layer(feat, operators=self.observable, symbol_names=self.symbols)
        f = tf.math.reduce_sum(f,axis=-1)
        return f

    def train_step(self, data):
        x,y=data
        with tf.GradientTape() as tape:
            tape.watch(self.trainable_variables)
            preds = self(x,training=True)
            energy = self.compiled_loss(preds, preds) #notice that compiled loss takes care only about the preds
        train_vars = self.trainable_variables
        grads=tape.gradient(energy,train_vars)

        self.cost_value.update_state(energy)
        return {k.name:k.result() for k in self.metrics}
